- 한국어 데이터로 단어문서행렬(TDM)을 만들어보자.

# 데이터 다운로드

- Naver Sentiment Movie Corpus (네이버 영화평) 데이터를 다운로드한다.

In [1]:
import requests

res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')

with open('../data/nsmc_train.csv', 'wb') as f:
    f.write(res.content)

# 데이터 불러오기

- 다운받은 데이터를 불러들인다.

In [2]:
import pandas as pd
nsmc = pd.read_csv('../data/nsmc_train.csv', sep='\t')

- .head() 명령어로 데이터의 앞부분에 위치한 5개 영화평을 살펴봅니다.

In [3]:
nsmc.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# 명사 추출 함수

- stanza로 이용해 명사만 추출하는 함수를 만든다.

In [4]:
import stanza

In [5]:
nlp = stanza.Pipeline('ko')

2021-03-26 11:37:05 INFO: Loading these models for language: ko (Korean):
| Processor | Package |
-----------------------
| tokenize  | kaist   |
| pos       | kaist   |
| lemma     | kaist   |
| depparse  | kaist   |

2021-03-26 11:37:05 INFO: Use device: gpu
2021-03-26 11:37:05 INFO: Loading: tokenize
2021-03-26 11:37:09 INFO: Loading: pos
2021-03-26 11:37:09 INFO: Loading: lemma
2021-03-26 11:37:09 INFO: Loading: depparse
2021-03-26 11:37:10 INFO: Done loading processors!


In [16]:
text = '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'

In [17]:
doc = nlp(text)

In [18]:
for sentence in doc.sentences:
    for word in sentence.words:
        lemma = word.lemma.split('+')
        xpos = word.xpos.split('+')
        for lem, pos in zip(lemma, xpos):
            if pos.startswith('n'):
                print(lem)

교도소
이야기구먼
..솔직히
재미
.평점
조정


In [6]:
def extract_nouns(text):
    doc = nlp(text)
    for sentence in doc.sentences:
        for word in sentence.words:
            lemma = word.lemma.split('+')
            xpos = word.xpos.split('+')
            for lem, pos in zip(lemma, xpos):
                if pos.startswith('n'):
                    yield lem

- nsmc의 3번 행, document 열의 내용을 확인한다.

In [7]:
nsmc.loc[3, 'document']

'교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'

- 위에서 정의한 extract_nouns 함수로 문장에서 명사를 추출해본다.

In [8]:
list(extract_nouns(nsmc.loc[3, 'document']))

['교도소', '이야기구먼', '..솔직히', '재미', '.평점', '조정']

# TDM 만들기

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

- max_features 추출할 단어의 개수는 100개로 설정한다. 이 경우, 빈도 순으로 최대 100 단어까지 포함하게 된다. 
- tokenizer에 extract_nouns함수를 지정하면, 한국어 명사만 추출하여 TDM을 만든다. 지정을 하지 않으면 빈칸 단위로 구분

In [10]:
cv = CountVectorizer(max_features=100, tokenizer=extract_nouns)

- 101개의 영화평을 대상으로 분석하고, 그 결과를 tdm에 저장한다.

In [11]:
tdm = cv.fit_transform(nsmc.loc[0:100, 'document']) # loc에서는 100을 포함

- tdm.shape로 tdm 결과 형태를 확인한다.

In [12]:
tdm.shape

(101, 100)

- 단어별 빈도를 표로 정리한다.

In [13]:
word_count = pd.DataFrame({
    '단어': cv.get_feature_names(),
    '빈도': tdm.sum(axis=0).flat
})

- 빈도순으로 정렬하여 확인한다.

In [14]:
word_count.sort_values('빈도', ascending=False).head()

,단어,빈도
47,영화,19
46,연기,6
49,완전,6
9,내용,5
92,진짜,5


- 빈도표를 저장한다.

In [15]:
word_count.to_excel('../data/nsmc-count.xlsx')